<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/master/fine-tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [59]:
import pandas as pd
import numpy as np

In [60]:
!wget -q --show-progress 'https://raw.githubusercontent.com/ebi-shirinbegi/The-HazNERC/main/train.txt'
!wget -q --show-progress 'https://raw.githubusercontent.com/ebi-shirinbegi/The-HazNERC/main/test.txt'
!wget -q --show-progress 'https://raw.githubusercontent.com/ebi-shirinbegi/The-HazNERC/main/dev.txt'
!mkdir data && mv train.txt data && mv test.txt data && mv dev.txt data

train.txt           100%[===================>]   2.13M  10.7MB/s    in 0.2s    
test.txt            100%[===================>] 108.65K  --.-KB/s    in 0.07s   
dev.txt             100%[===================>] 161.38K  --.-KB/s    in 0.08s   
mkdir: cannot create directory ‘data’: File exists


In [61]:
!cat data/train.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > train.txt.tmp
!cat data/test.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > test.txt.tmp
!cat data/dev.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > dev.txt.tmp

In [62]:
!git clone https://github.com/huggingface/transformers

fatal: destination path 'transformers' already exists and is not an empty directory.


In [63]:
!pip -q install transformers/
!pip -q install sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [64]:
!wget "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"

--2024-02-20 16:09:49--  https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 991 [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]     991  --.-KB/s    in 0s      

2024-02-20 16:09:50 (60.4 MB/s) - ‘preprocess.py’ saved [991/991]



In [65]:
!python3 preprocess.py dev.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > dev.txt
!python3 preprocess.py train.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > train.txt
!python3 preprocess.py test.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > test.txt

In [66]:
!pip -q install -r transformers/examples/pytorch/token-classification/requirements.txt

In [67]:
!pip -q install conllu

In [68]:
!mkdir processed_data
!mv train.txt processed_data/ && mv test.txt processed_data/ && mv dev.txt processed_data/

mkdir: cannot create directory ‘processed_data’: File exists


In [69]:
!cat processed_data/train.txt processed_data/test.txt processed_data/dev.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt

In [70]:
!python3 transformers/examples/pytorch/token-classification/run_ner.py

2024-02-20 16:10:30.649501: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-20 16:10:30.649555: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-20 16:10:30.650915: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-20 16:10:31.919760: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
usage: run_ner.py [-h] --model_name_or_path MODEL_NAME_OR_PATH [--config_name CONFIG_NAME]
                  [--tokenizer_name TOKENIZER_NAME] [--cache_dir CACHE_DIR]
                  [--model_revision MODEL_REVISION] [--token TOKEN]
      

In [71]:
!python3 transformers/examples/legacy/token-classification/run_ner.py \
--data_dir ./processed_data/ \
--labels labels.txt \
--model_name_or_path HooshvareLab/bert-base-parsbert-uncased \
--output_dir eval/ \
--max_seq_length  256 \
--per_device_train_batch_size 8 \
--save_steps 1000 \
--save_total_limit 1 \
--eval_steps 100 \
--logging_strategy steps \
--logging_steps 100 \
--logging_first_step True \
--evaluation_strategy steps \
--seed 1 \
--num_train_epochs 5\
--do_train \
--do_eval \
--do_predict

2024-02-20 16:10:41.181764: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-20 16:10:41.181817: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-20 16:10:41.183039: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-20 16:10:42.311388: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
02/20/2024 16:10:44 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: True, 16-bits training: False
02/20/2024 16:10:44 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
acc

In [72]:
!ls -lh

total 4.7M
drwxr-xr-x  2 root root 4.0K Feb 20 14:47 data
-rw-r--r--  1 root root 697K Feb 20 16:07 data.zip
-rw-r--r--  1 root root 162K Feb 20 16:09 dev.txt.tmp
drwx------  5 root root 4.0K Feb 20 15:37 drive
drwxr-xr-x  4 root root 4.0K Feb 20 16:43 eval
-rw-r--r--  1 root root   22 Feb 20 15:46 eval.zip
-rw-r--r--  1 root root   74 Feb 20 16:10 labels.txt
-rw-r--r--  1 root root  991 Feb 20 16:09 preprocess.py
drwxr-xr-x  2 root root 4.0K Feb 20 16:10 processed_data
-rw-r--r--  1 root root 1.6M Feb 20 16:07 processed_data.zip
-rw-r--r--  1 root root   22 Feb 20 15:55 runs.zip
drwxr-xr-x  1 root root 4.0K Feb 14 14:28 sample_data
-rw-r--r--  1 root root 109K Feb 20 16:09 test.txt.tmp
-rw-r--r--  1 root root 2.2M Feb 20 16:09 train.txt.tmp
drwxr-xr-x 16 root root 4.0K Feb 20 14:48 transformers


In [73]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
!zip -r data.zip data
!zip -r processed_data.zip processed_data

updating: data/ (stored 0%)
updating: data/test.txt (deflated 71%)
updating: data/train.txt (deflated 72%)
updating: data/dev.txt (deflated 71%)
updating: processed_data/ (stored 0%)
updating: processed_data/test.txt (deflated 71%)
updating: processed_data/cached_train_BertTokenizer_256.lock (stored 0%)
updating: processed_data/cached_dev_BertTokenizer_256 (deflated 95%)
updating: processed_data/cached_test_BertTokenizer_256.lock (stored 0%)
updating: processed_data/cached_test_BertTokenizer_256 (deflated 95%)
updating: processed_data/train.txt (deflated 72%)
updating: processed_data/dev.txt (deflated 71%)
updating: processed_data/cached_dev_BertTokenizer_256.lock (stored 0%)
updating: processed_data/cached_train_BertTokenizer_256 (deflated 95%)


In [75]:
!mv dev.txt.tmp drive/MyDrive
!mv labels.txt drive/MyDrive
!mv preprocess.py drive/MyDrive
!mv test.txt.tmp drive/MyDrive
!mv train.txt.tmp drive/MyDrive